In [2]:
from btree import BTreeNode, BTree

In [53]:
from __future__ import annotations
from typing import List, Optional, Tuple, Iterator, TypeVar, Generic
from bisect import bisect_left
from collections import deque
import logging

K = TypeVar('K')

class BTreeChild(BTree[K]):
    """
    Clase que representa un nodo hijo en un Árbol B.
    Hereda de BTreeNode y puede tener funcionalidades adicionales.
    """
    def show(self, level: int = 0) -> None:
        self._show(self.root)

    def _show(self, node: BTreeNode[K], level: int = 0) -> None:
        if node is None:
            return
        print("  " * level + f"Node: {node.keys}")
        for child in node.children:
            self._show(child, level + 1)

    def show_in_order(self) -> List[K]:
        """
        Muestra los elementos del árbol en orden.
        :return: Lista de elementos en orden.
        """
        result = []
        self._show_in_order(self.root, result)
        return result

    def _show_in_order(self, node: BTreeNode[K], result: List[K]) -> None:
        if node is None:
            return
        for i, key in enumerate(node.keys):
            if not node.leaf :
                self._show_in_order(node.children[i], result)
            result.append(key)
        if not node.leaf:
            self._show_in_order(node.children[len(node.keys)], result)
            
    def nearest_element(self, key: K) -> Optional[K]:
        if self.root is None:
            return None
        return self._nearest_element(self.root, key, float('inf'))
        
    def _nearest_element(self, node: BTreeNode[K], key: K,min_dis: K)-> Optional[K]:
        if node is None:
            return None
        
        ret = None
        
        pos = bisect_left(node.keys, key)
        cur_dis  = abs(node.keys[pos] - key) if pos < len(node.keys) else float('inf')
        
        if cur_dis < min_dis:
            min_dis = cur_dis
            ret = node.keys[pos]
            
        cur_dis = abs(node.keys[pos+1] - key) if pos + 1 < len(node.keys) else float('inf')
        if cur_dis < min_dis:
            min_dis = cur_dis
            ret = node.keys[pos + 1]

        option2 = self._nearest_element(node.children[pos] if pos < len(node.children) else None, key, min_dis)
        if option2 is not None:
            if ret is not None:
                if abs(option2 - key) < abs(ret - key):
                    ret = option2
            else :
                ret = option2
                
        return ret          

In [54]:
tree = BTreeChild[int](3)
values = [10, 20, 5, 17, 2, 3, 100, -1]
for v in values:
    tree.insert(v)
assert tree.traverse() == sorted(values)
assert tree.validate()
tree.show()

Node: [10]
  Node: [-1, 2, 3, 5]
  Node: [17, 20, 100]


In [55]:
print(tree.show_in_order())

[-1, 2, 3, 5, 10, 17, 20, 100]


In [56]:
target = 15
print(tree.nearest_element(target))

17


1. El tiempo amortizado es de ``log(n)`` debido a que en la recursion solo iteramos hasta la altura del arbol.
2. Borrados mal gestionados 
    2.1 Caso en el que solo se borra y no se remplaza por algun valor que conserve la invariaza.

$$
        |4|10|100|                   \qquad  \qquad \qquad  \qquad \qquad \qquad \qquad \qquad \qquad \qquad                   |4|\qquad |100| \\

|-1|3|  |6|9|    |120|200|      ----borramos 10------>|-1|3|  |6|9|    |120|200|
$$
Al realizar ``pos = bisect_left``, tendriamos que mandar la recursion sobre pos, (pos-1, pos-2, ....), si desde [pos-1, pos-2, pos-3, ....] son vacios.